# Project Description

You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Compute the moving average of how much the customer paid in a seven days window (i.e., current day + 6 days before). `average_amount` should be rounded to two decimal places.

Return the result table ordered by `visited_on` in ascending order.

The result format is in the following example.

## Example

### Input: 

**Customer table:**
| customer_id | name         | visited_on   | amount      |
|-------------|--------------|--------------|-------------|
| 1           | Jhon         | 2019-01-01   | 100         |
| 2           | Daniel       | 2019-01-02   | 110         |
| 3           | Jade         | 2019-01-03   | 120         |
| 4           | Khaled       | 2019-01-04   | 130         |
| 5           | Winston      | 2019-01-05   | 110         | 
| 6           | Elvis        | 2019-01-06   | 140         | 
| 7           | Anna         | 2019-01-07   | 150         |
| 8           | Maria        | 2019-01-08   | 80          |
| 9           | Jaze         | 2019-01-09   | 110         | 
| 1           | Jhon         | 2019-01-10   | 130         | 
| 3           | Jade         | 2019-01-10   | 150         | 

### Output: 
| visited_on   | amount       | average_amount |
|--------------|--------------|----------------|
| 2019-01-07   | 860          | 122.86         |
| 2019-01-08   | 840          | 120.00         |
| 2019-01-09   | 840          | 120.00         |
| 2019-01-10   | 1000         | 142.86         |

# Intuition

The problem requires calculating the cumulative total amount spent over a 7-day period and the average of this total. We need to consider each day's data along with the previous 6 days to get our 7-day window.

# Approach

1. **Group by Date:** First, aggregate the amounts by date to get daily totals.
2. **Use Window Functions:** 
   - Use `SUM` with an `OVER` clause to calculate the cumulative sum over the last 7 days.
   - Use `AVG` similarly to compute the 7-day average.
3. **Offset:** Since we need a full 7 days of data for each row, we'll offset our result set to start from the 7th day onward. You have to use SUM, AVG, ROUND and a special line like 
OVER(ORDER BY visited_on ROWS BETWEEN 6 AND CURRENT ROW)

# Complexity

- **Time complexity:** $$O(n \log n)$$ due to sorting for window functions where \( n \) is the number of rows in the `Customer` table.
- **Space complexity:** $$O(n)$$ for creating a temporary table with all unique dates.

# Code

```sql
WITH TEMP_TABLE AS 
    (SELECT visited_on,
            SUM(amount) AS daily_amount
     FROM Customer
     GROUP BY visited_on)

SELECT visited_on,
       SUM(daily_amount) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS amount,
       ROUND(AVG(daily_amount) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) AS average_amount
FROM TEMP_TABLE
ORDER BY visited_on
LIMIT 1000 OFFSET 6;